In [1]:
import json

with open('C:/Users/Drenaline/Documents/nosql-challenge/Resources/establishments.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
   
from pprint import pprint
from pymongo import MongoClient


In [2]:
import pandas as pd

In [3]:

client = MongoClient('localhost', 27017)

In [4]:
db = client['uk_food']
collection = db['establishments']

document_count = collection.count_documents({})
print(f"Number of documents in the collection: {document_count}")

Number of documents in the collection: 77571


In [5]:
if isinstance(data, list):
    collection.insert_many(data)
else:
    print("Data is not in the expected format (a list of documents)")
    
document_count = collection.count_documents({})
print(f"Number of documents in the collection: {document_count}")

for doc in collection.find().limit(5):
    print(doc)

Number of documents in the collection: 117350
{'_id': ObjectId('654fca8aa7bd6cfec09977e3'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('1.195625'), 'latitude': Decimal128('51.083812')}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataS

In [6]:
query = {'BusinessName': 'The Tea Room'}
document = collection.find_one(query)

if document:
    print("Found a document:", document)
else:
    print("No document matches the query.")

Found a document: {'_id': ObjectId('654fca8aa7bd6cfec09988b7'), 'FHRSID': 1219650, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000097935', 'BusinessName': 'The Tea Room', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'Perry Court Farm', 'AddressLine2': 'Canterbury Road', 'AddressLine3': 'Boughton Aluph', 'AddressLine4': 'Kent', 'PostCode': 'TN25 4ES', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2021-11-18T00:00:00', 'LocalAuthorityCode': '179', 'LocalAuthorityName': 'Ashford', 'LocalAuthorityWebSite': 'http://www.ashford.gov.uk', 'LocalAuthorityEmailAddress': 'envhealth@ashford.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 10, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('0.911231'), 'latitude': Decimal128('51.187073')}, 'RightToReply': '', 'Distance': 4605.994313264293, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'i

In [7]:
establishments_collection = db['establishments']

In [8]:
new_establishment = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

insert_result = establishments_collection.insert_one(new_establishment)
print(f"Inserted document ID: {insert_result.inserted_id}")

Inserted document ID: 654fe684a7ab266a67875c44


In [9]:
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
projection = {'BusinessTypeID': 1, '_id': 0}  
document = collection.find_one(query, projection)

if document:
    print("Found a document:", document)
    business_type_id = document.get('BusinessTypeID')
else:
    print("No document matches the query.")

Found a document: {'BusinessTypeID': 1}


In [10]:
update_query = {'BusinessName': 'Penang Flavours'}
new_values = {"$set": {'BusinessTypeID':1}}

update_result = collection.update_one(update_query, new_values)

if update_result.modified_count > 0:
    print("Document updated successfully.")
else:
    print("No document was updated.")

No document was updated.


In [11]:
dover_count = collection.count_documents({'LocalAuthorityName': 'Dover'})
print(f"Number of establishments in Dover: {dover_count}")

delete_result = collection.delete_many({'LocalAuthorityName': 'Dover'})
print(f"Deleted {delete_result.deleted_count} establishments from Dover.")

post_delete_count = collection.count_documents({})
print(f"Total number of establishments after deletion: {post_delete_count}")

Number of establishments in Dover: 994
Deleted 994 establishments from Dover.
Total number of establishments after deletion: 116357


In [12]:
update_lat_long = collection.update_many(
    {}, 
    [
        {"$set": {"geocode.latitude": {"$toDecimal": "$geocode.latitude"}}},
        {"$set": {"geocode.longitude": {"$toDecimal": "$geocode.longitude"}}}
    ]
)

print(f"Updated {update_lat_long.modified_count} documents with latitude and longitude.")

Updated 38786 documents with latitude and longitude.


In [13]:
import re

def is_integer(s):
    if isinstance(s, str):
        return re.fullmatch(r"-?\d+", s) is not None
    return False  # Return False if s is not a string

# Iterate over the documents and update
for doc in collection.find({"RatingValue": {"$exists": True}}):
    rating_value = doc.get('RatingValue', '')
    
    # Check if rating_value is a string that can be converted to an integer
    if is_integer(rating_value):
        new_rating_value = int(rating_value)
        collection.update_one({'_id': doc['_id']}, {'$set': {'RatingValue': new_rating_value}})

In [14]:
#answering the 1st of 4 questions!


# Query for hygiene score equal to 20
query = {"scores.Hygiene": 20}
count = collection.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document
first_document = collection.find_one(query)
pprint(first_document)

# Convert to DataFrame
df = pd.DataFrame(list(collection.find(query)))
print(f"Number of rows in DataFrame: {len(df)}")
print(df.head(10))

Number of establishments with a hygiene score of 20: 123
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('654fca8aa7bd6cfec0999315'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establi

In [15]:
query = {"LocalAuthorityName": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}}
count = collection.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

first_document = collection.find_one(query)
pprint(first_document)

df = pd.DataFrame(list(collection.find(query)))
print(f"Number of rows in DataFrame: {len(df)}")
print(df.head(10))

Number of establishments in London with RatingValue >= 4: 99
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('654fca8ba7bd6cfec099ad30'),
 'geocode': {'latitude': Decimal128('51.369321'),
             'longitude': Decimal128('0.508551')},
 '

In [16]:
# Geocode for "Penang Flavours"
latitude = 51.49014200
longitude = 0.08384000

# Query for top 5 establishments with RatingValue of 5, sorted by lowest hygiene score, nearest to "Penang Flavours"
query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - 0.01, "$lte": latitude + 0.01},
    "geocode.longitude": {"$gte": longitude - 0.01, "$lte": longitude + 0.01}
}
count = collection.count_documents(query)
print(f"Number of establishments near 'Penang Flavours' with RatingValue of 5: {count}")

import pandas as pd

df = pd.DataFrame(list(collection.find(query).sort("scores.Hygiene", 1).limit(5)))
print(f"Number of rows in DataFrame: {len(df)}")
print(df.head(10))

Number of establishments near 'Penang Flavours' with RatingValue of 5: 261
Number of rows in DataFrame: 5
                        _id   FHRSID  ChangesByServerID  \
0  654fca8ba7bd6cfec099eb74   694609                  0   
1  654fca8ba7bd6cfec099eb8c   695241                  0   
2  654fca8ba7bd6cfec099eb8f   694478                  0   
3  654fca8ba7bd6cfec099eb47   695223                  0   
4  654fca8ba7bd6cfec099eb57  1380578                  0   

  LocalAuthorityBusinessID                         BusinessName  \
0             PI/000116619                            Volunteer   
1             PI/000179088              Plumstead Manor Nursery   
2             PI/000086506                    Atlantic Fish Bar   
3             PI/000178842                              Iceland   
4                    14425  Howe and Co Fish and Chips - Van 17   

                            BusinessType  BusinessTypeID  \
0                      Pub/bar/nightclub            7843   
1               

In [17]:
# Aggregation query
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

results = list(collection.aggregate(pipeline))
df = pd.DataFrame(results)
print(df)

               _id  count
0           Thanet   3390
1        Greenwich   2646
2        Maidstone   2139
3           Newham   2133
4            Swale   2058
5       Chelmsford   2040
6           Medway   2016
7           Bexley   1821
8  Southend-On-Sea   1758
9         Tendring   1626
